# AlexNet



In [1]:
import time
import torch
from torch import nn,optim
import torchvision

import sys
sys.path.append("..")
import d2lzh_pytorch as d2l
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class AlexNet(nn.Module):
    def __init__(self):
        super(AlexNet,self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1,96,11,4),
            nn.ReLU(),
            nn.MaxPool2d(3,2),
            nn.Conv2d(96,256,5,1,2),
            nn.ReLU(),
            nn.MaxPool2d(3,2),
            nn.Conv2d(256,384,3,1,1),
            nn.ReLU(),
            nn.Conv2d(384,384,3,1,1),
            nn.ReLU(),
            nn.Conv2d(384,256,3,1,1),
            nn.ReLU(),
            nn.MaxPool2d(3,2)
        )
        self.fc = nn.Sequential(
            nn.Linear(256*5*5,4096),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(4096,4096),
            nn.ReLU(),
            nn.Dropout(0.5),
            # 输出层。由于这里使用Fashion-MNIST，所以用类别数为10，而非论文中的1000
            nn.Linear(4096, 10),
        
        )
    def forward(self, img):
        feature = self.conv(img)
        output = self.fc(feature.view(img.shape[0], -1))
        return output
    

net = AlexNet()
print(net)

D:\Anaconda\envs\torch\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
D:\Anaconda\envs\torch\lib\site-packages\numpy\.libs\libopenblas.JPIJNSWNNAN3CE6LLI5FWSPHUT2VXMTH.gfortran-win_amd64.dll
D:\Anaconda\envs\torch\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  stacklevel=1)


AlexNet(
  (conv): Sequential(
    (0): Conv2d(1, 96, kernel_size=(11, 11), stride=(4, 4))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(96, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(256, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU()
    (8): Conv2d(384, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU()
    (10): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU()
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Sequential(
    (0): Linear(in_features=6400, out_features=4096, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.5, inplace=False)
    (3): Linear(in_features=4096, out_features=4096, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.5, inplace=False)
    (

# 读取数据

读取数据的时候额外做了一步将图像高和宽扩大到AlexNet使用的图像高和宽224。

这个可以通过`torchvision.transforms.Resize`实例来实现。也就是说，我们在ToTensor实例前使用Resize实例，然后使用`Compose实例`来将这两个变换串联以方便调用。

In [2]:
def load_data_fashion_mnist(batch_size,
                            resize = None,
                            root = "~/Datasets/FashionMNIST"):
    trans=[]
    if resize:
        trans.append(torchvision.transforms.Resize(size = resize))
    trans.append(torchvision.transforms.ToTensor())
    transform = torchvision.transforms.Compose(trans)
    mnist_train = torchvision.datasets.FashionMNIST(root = root,
                                                    train = True,
                                                    download = True,
                                                   transform =transform)
    mnist_test = torchvision.datasets.FashionMNIST(root = root,
                                                    train = False,
                                                    download = True,
                                                   transform =transform)
    
    train_iter = torch.utils.data.DataLoader(mnist_train, batch_size=batch_size, shuffle=True, num_workers=12)
    test_iter = torch.utils.data.DataLoader(mnist_test, batch_size=batch_size, shuffle=False, num_workers=12)

    return train_iter, test_iter

batch_size = 128
# 如出现“out of memory”的报错信息，可减小batch_size或resize
train_iter, test_iter = load_data_fashion_mnist(batch_size, resize=224,root = "D:\JupyterFile\Machine_Learning\Pytorch\Datasets\FashionMNIST")

lr,num_epochs = 0.001,5
optimizer = torch.optim.Adam(net.parameters(),lr = lr)
d2l.train_ch5(net,train_iter,test_iter,batch_size,optimizer,device,num_epochs)

training on  cuda


  0%|                                                                                          | 0/469 [00:00<?, ?it/s]

epoch 1, loss 0.6426, train acc 0.756, test acc 0.859, time 66.9 sec


  0%|                                                                                          | 0/469 [00:00<?, ?it/s]

epoch 2, loss 0.3414, train acc 0.872, test acc 0.861, time 63.8 sec


...

epoch 3, loss 0.2992, train acc 0.888, test acc 0.888, time 63.8 sec


  0%|                                                                                          | 0/469 [00:00<?, ?it/s]

epoch 4, loss 0.2708, train acc 0.898, test acc 0.892, time 63.8 sec


100%|████████████████████████████████████████████████████████████████████████████████| 469/469 [00:55<00:00,  8.47it/s]


epoch 5, loss 0.2494, train acc 0.907, test acc 0.904, time 64.0 sec
